In [72]:
import pandas as pd
import os
import re

# 센서 데이터와 레이블 데이터 경로 설정
sensor_data_path = '../data/sensor_data'
label_data_path = '../data/label_data'

# 센서 데이터 로드 및 전처리
sensor_data = pd.DataFrame()
for root, dirs, files in os.walk(sensor_data_path):
    for file in files:
        if file.endswith('.csv'):
            file_path = os.path.join(root, file)
            df = pd.read_csv(file_path)
            subject_id = file.split('T')[0]  # 파일명에서 주체 ID 추출
            task_code = re.sub(r'\D', '', file.split('T')[1].split('R')[0])  # 파일명에서 태스크 코드 추출 (숫자만 추출)
            trial_id = file.split('R')[1].split('.')[0]  # 파일명에서 트라이얼 ID 추출
            df['Subject ID'] = subject_id
            df['Task Code'] = task_code
            df['Trial ID'] = trial_id
            sensor_data = pd.concat([sensor_data, df], ignore_index=True)

# 레이블 데이터 로드 및 전처리
label_data = pd.DataFrame()
for root, dirs, files in os.walk(label_data_path):
    for file in files:
        if file.endswith('.xlsx'):
            file_path = os.path.join(root, file)
            df = pd.read_excel(file_path)
            subject_id = file.split('_')[0]  # 파일명에서 주체 ID 추출
            df['Subject ID'] = subject_id
            df.rename(columns={'Task Code (Task ID)': 'Task Code'}, inplace=True)  # 컬럼명 수정
            label_data = pd.concat([label_data, df], ignore_index=True)

# 결측값 처리
label_data[['Description', 'Task Code']] = label_data[['Description', 'Task Code']].fillna(method='ffill')

# 데이터 타입 변환
sensor_data['Subject ID'] = sensor_data['Subject ID'].astype(str)
sensor_data['Task Code'] = sensor_data['Task Code'].astype(int)
sensor_data['Trial ID'] = sensor_data['Trial ID'].astype(int)
label_data['Subject ID'] = label_data['Subject ID'].astype(str)
label_data['Task Code'] = label_data['Task Code (Task ID)'].str.extract(r'(\d+)').astype(int)  # 숫자만 추출하여 정수로 변환
label_data['Trial ID'] = label_data['Trial ID'].astype(int)

# 센서 데이터와 레이블 데이터 결합
merged_data = pd.merge(sensor_data, label_data, on=['Subject ID', 'Task Code', 'Trial ID'])

# 결과 출력
print(merged_data.head())
print(label_data)


Empty DataFrame
Columns: [TimeStamp(s), FrameCounter, AccX, AccY, AccZ, GyrX, GyrY, GyrZ, EulerX, EulerY, EulerZ, Subject ID, Task Code, Trial ID, Description, Fall_onset_frame, Fall_impact_frame]
Index: []
      Task Code                                   Description  Trial ID  \
0             1          Forward fall when trying to sit down         1   
1             1          Forward fall when trying to sit down         2   
2             1          Forward fall when trying to sit down         3   
3             1          Forward fall when trying to sit down         4   
4             1          Forward fall when trying to sit down         5   
...         ...                                           ...       ...   
2341         15  Backward fall while walking caused by a slip         1   
2342         15  Backward fall while walking caused by a slip         2   
2343         15  Backward fall while walking caused by a slip         3   
2344         15  Backward fall while walkin